# Exemple basique utilisation geoparquet

## 1. La base dvf

In [ ]:
import subprocess

subprocess.call("mc cp s3/projet-formation/nouvelles-sources/data/geoparquet/dvf.parquet dvf.parquet", shell=True)
subprocess.call("mc cp s3/projet-formation/nouvelles-sources/data/geoparquet/carreaux.parquet carreaux.parquet", shell=True)

In [2]:
from cartiflette import carti_download

# 1. Fonds communaux
contours_villes_arrt = carti_download(
    values = ["75", "92", "93", "94"],
    crs = 4326,
    borders="COMMUNE_ARRONDISSEMENT",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

# 2. Départements 
departements = contours_villes_arrt.dissolve("INSEE_DEP").reset_index()
#buffer_75 = departements.loc[departements["INSEE_DEP"] == "75"].to_crs(2154).buffer(distance = 2000).plot()

## Exploitation

In [ ]:
import duckdb
duckdb.execute("INSTALL spatial;")
duckdb.execute("LOAD spatial;")

In [35]:
reference_lon = 2.35  # Replace with your reference longitude
reference_lat = 48.853 # Replace with your reference latitude


carreaux_duckdb = duckdb.sql(f"""
FROM read_parquet('carreaux_geoparquet.parquet')
SELECT
    *, ST_AsText(geometry) AS geom_text
WHERE st_distance(
        st_centroid(geometry),
        ST_Transform(st_point({reference_lat}, {reference_lon}), 'EPSG:4326', 'EPSG:2154')
    ) / 1000 < 2
""")

In [36]:
carreaux = carreaux_duckdb.to_df()
carreaux = gpd.GeoDataFrame(carreaux)
carreaux = carreaux.drop("geometry", axis = "columns")
carreaux['geometry'] = gpd.GeoSeries.from_wkt(carreaux['geom_text'])
carreaux = carreaux.set_geometry('geometry', crs=2154).drop("geom_text", axis = "columns")

In [47]:
carreaux['prop_men_pauv'] = 100*carreaux['men_pauv']/carreaux['men']

In [ ]:
import folium
import branca.colormap as cm

carreaux = carreaux.to_crs(4326)
centroid = carreaux.geometry.unary_union.centroid
map_center = [centroid.y, centroid.x]

# Create a linear colormap
colormap = cm.LinearColormap(
    colors=['green', 'yellow', 'red'],  # Color range
    vmin=carreaux['prop_men_pauv'].min(),  # Minimum value of the variable
    vmax=carreaux['prop_men_pauv'].max(),  # Maximum value of the variable
    caption='Proportion ménages pauvres'  # Legend title
)

def style_function(feature):
    proportion = feature['properties']['prop_men_pauv']
    return {
        'fillColor': colormap(proportion),
        'color': 'black',         # Border color
        'weight': 1,              # Border thickness
        'fillOpacity': 0.7,       # Fill transparency
    }

m = folium.Map(location=map_center, zoom_start=14, tiles='cartodbpositron')

# Add GeoJson layer to the map with tooltips
folium.GeoJson(
    carreaux,
    name='carreaux Borders',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['idcar_200m', 'idcar_1km', 'idcar_nat', 'prop_men_pauv'],
        aliases=['ID Car 200m:', 'ID Car 1km:', 'ID Car Nat:', 'Proportion ménages pauvres:'],
        localize=True
    )
).add_to(m)

# Add the colormap legend to the map
colormap.add_to(m)

# Optional: Add layer control if you have multiple layers
folium.LayerControl().add_to(m)

# Display the map
m

In [ ]:
duckdb.sql("SELECT * FROM read_parquet('dvf_geoparquet.parquet') LIMIT 2")